# Predict velocity values using deep learning

This notebook trains and uses an LSTM to predict velocity values. Modified from the code in [Encord's](https://encord.com/blog/time-series-predictions-with-recurrent-neural-networks/) post. An LSTM uses a memory cell to store information that the model uses to predict future values. Input, forget, and output gates control what is stored in, removed, and output from the LSTM. The LSTM's memory makes it ideal for predicting time series values; the lookback must be modified to encapsulate enough days to predict but not too many for overfitting.

---

### 1. Load packages and open dataset

Choose a point (`upstream`, `middle`, `terminus`).

In [31]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

In [32]:
point_name = 'middle' # 'upstream', 'middle', or 'terminus'
input_file='../data/ai_ready/' + point_name + '_timeseries.csv'

# load the dataset, drop the series column
df = pd.read_csv(input_file)

df.drop(columns="series", inplace=True)
dataset=df['velocity'].values.reshape(-1, 1)
dataset.shape

(364, 1)

### 2. Preprocessing

Here, we convert the values into a matrix, normalize, and split into train and test data. Normalization ensures that predictions are scaled properly so that one data point does not dominate training or predictions. We split into 80/20 train/test data and use a look back of 15, meaning that the model will consider the previous 15 values to predict the next value.

In [33]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	print(len(dataX), len(dataY))
	return np.array(dataX), np.array(dataY)

In [34]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
dataset.shape

(364, 1)

In [35]:
# split into train and test sets, 70% test data, 30% training data
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
train.shape, test.shape

((254, 1), (110, 1))

In [36]:
# reshape into X=t and Y=t+1, timestep 5
look_back = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

trainX = trainX[:-1]
trainY = trainY[:-1]
trainX.shape, trainY.shape, testX.shape, testY.shape

248 248
104 104


((247, 5), (247,), (104, 5), (104,))

### 3. Train and test

Create the model and train for 5000 epochs. Save it to the `models` folder.

In [37]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

trainX.shape, trainY.shape, testX.shape, testY.shape

((247, 5, 1), (247,), (104, 5, 1), (104,))

In [38]:
# create and fit the LSTM network, optimizer=adam, 50 neurons, 2 LSTM layers
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(look_back, 1)))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, trainY, epochs=500, batch_size=32, validation_data=(testX, testY), verbose=2)

Epoch 1/500


/Users/cjense/miniconda3/envs/mlgeo-jensencc/lib/python3.10/site-packages/keras/layers/rnn/rnn.py:205: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



8/8 - 2s - 258ms/step - loss: 0.2051 - val_loss: 0.0869
Epoch 2/500
8/8 - 0s - 8ms/step - loss: 0.0466 - val_loss: 0.0238
Epoch 3/500
8/8 - 0s - 10ms/step - loss: 0.0287 - val_loss: 0.0429
Epoch 4/500
8/8 - 0s - 10ms/step - loss: 0.0303 - val_loss: 0.0149
Epoch 5/500
8/8 - 0s - 9ms/step - loss: 0.0209 - val_loss: 0.0116
Epoch 6/500
8/8 - 0s - 8ms/step - loss: 0.0214 - val_loss: 0.0193
Epoch 7/500
8/8 - 0s - 8ms/step - loss: 0.0202 - val_loss: 0.0129
Epoch 8/500
8/8 - 0s - 8ms/step - loss: 0.0201 - val_loss: 0.0130
Epoch 9/500
8/8 - 0s - 8ms/step - loss: 0.0181 - val_loss: 0.0188
Epoch 10/500
8/8 - 0s - 8ms/step - loss: 0.0175 - val_loss: 0.0119
Epoch 11/500
8/8 - 0s - 9ms/step - loss: 0.0203 - val_loss: 0.0141
Epoch 12/500
8/8 - 0s - 10ms/step - loss: 0.0198 - val_loss: 0.0150
Epoch 13/500
8/8 - 0s - 9ms/step - loss: 0.0187 - val_loss: 0.0122
Epoch 14/500
8/8 - 0s - 10ms/step - loss: 0.0186 - val_loss: 0.0151
Epoch 15/500
8/8 - 0s - 9ms/step - loss: 0.0204 - val_loss: 0.0133
Epoch 16/5

In [39]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [40]:
model.save('../models/lstm_model.keras')

In [41]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

### 4. Analyze and plot predictions

Calculate RMSE for train and test, shift for plotting, and plot using Plotly. Note that the first 15 timesteps of both train and test are not accounted for in the prediction data since they were used to train the model.

In [42]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 37.09 RMSE
Test Score: 42.97 RMSE


In [43]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict) + look_back, :] = trainPredict

In [44]:
# shift test predictions for plotting
testPredictPlot= np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-2, :] = testPredict

In [45]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(y=scaler.inverse_transform(dataset).flatten(), mode='lines', name='Actual Data'))
fig.add_trace(go.Scatter(y=trainPredictPlot.flatten(), mode='lines', name='Train Predictions'))
fig.add_trace(go.Scatter(y=testPredictPlot.flatten(), mode='lines', name='Test Predictions'))

# Update layout for dark theme
fig.update_layout(title='LSTM Predictions vs Actual Data for ' + point_name.capitalize() + ' Point', xaxis_title='Time', yaxis_title='Velocity')
# Update x-axis to show only one tick per year
years = pd.to_datetime(df['time']).dt.year
unique_years = sorted(years.unique())
fig.update_xaxes(tickvals=[df.index[years == year][0] for year in unique_years], ticktext=unique_years)
fig.write_image('../figures/8_LSTM_preds_' + point_name + '.png')
fig.show()